In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [0]:
dbutils.widgets.text("load_flag"," ")

In [0]:
load_flag=int(dbutils.widgets.get("load_flag"))

In [0]:
df=spark.sql("select *from oms_catalog.silver.customers_silver")

In [0]:
if load_flag==0:
  df_old=spark.sql('''select  customer_id,dim_customer_key,create_date,update_date from oms_catalog.gold.dim_customer''')
  
else:
    df_old=spark.sql('''select 0 customer_id, 0 dim_customer_key,0 create_date, 0 update_date from oms_catalog.silver.customers_silver where 1=0''')

In [0]:
df_old=df_old.withColumnRenamed("customer_id","customer_id_old")

In [0]:
df_join=df.join(df_old,df.customer_id==df_old.customer_id_old,how='left')

In [0]:
df_old=df_join.filter(df_join["dim_customer_key"].isNotNull())

In [0]:
df_new=df_join.filter(df_join["dim_customer_key"].isNull())

In [0]:
df_old=df_old.drop("customer_id_old")
df_new=df_new.drop("customer_id_old")

In [0]:

df_old=df_old.withColumn("update_date",current_timestamp())
df_old=df_old.withColumn("create_date", to_timestamp("create_date"))

In [0]:
df_new=df_new.withColumn("create_date",current_timestamp())
df_new=df_new.withColumn("update_date",current_timestamp())

In [0]:
if load_flag==1:
    max_surrogate_key=1
  

else:
    df_max_surr=spark.sql("SELECT MAX(dim_customer_key) as max_surrogate_key FROM oms_catalog.gold.dim_customer")
    max_surrogate_key=df_max_surr.collect()[0][0]
   

In [0]:
df_new=df_new.withColumn("dim_customer_key",monotonically_increasing_id()+max_surrogate_key)

In [0]:
df_final=df_new.unionByName(df_old)

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("oms_catalog.gold.dim_customer"):
     dlt_obj=DeltaTable.forPath(spark,"abfss://gold@azureomsprojectusa.dfs.core.windows.net/dim_customer")
     dlt_obj.alias("trg").merge(df_final.alias("src"),"trg.dim_customer_key=src.dim_customer_key").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
   
else:
     df_final.write.mode("overwrite")\
        .format("delta")\
        .option("path","abfss://gold@azureomsprojectusa.dfs.core.windows.net/dim_customer")\
        .saveAsTable("oms_catalog.gold.dim_customer")
   
